In [1]:
import os
import glob
import pandas as pd
import numpy as np

In [2]:
df = pd.concat([pd.read_csv(file_) for file_ in glob.glob(os.path.join('../data/', '*'))], ignore_index=True)

### The Data

In [3]:
df.head().T

,0,1,2,3,4
year,2016,2016,2016,2016,2016
ocid,ocds-5wno2w-CE-20160000536478-586277,ocds-5wno2w-CE-20160000536479-586277,ocds-5wno2w-CE-20160000536480-586277,ocds-5wno2w-CE-20160000536481-586277,ocds-5wno2w-CE-20160000536482-586277
date,2016-05-16T00:00:00-05:00,2016-05-16T00:00:00-05:00,2016-05-16T00:00:00-05:00,2016-05-16T00:00:00-05:00,2016-05-16T00:00:00-05:00
region,CHIMBORAZO,CHIMBORAZO,CHIMBORAZO,CHIMBORAZO,CHIMBORAZO
title,Orden de compra CE-20160000536478,Orden de compra CE-20160000536479,Orden de compra CE-20160000536480,Orden de compra CE-20160000536481,Orden de compra CE-20160000536482
description,Orden de compra para adquirir los siguientes ...,Orden de compra para adquirir los siguientes ...,Orden de compra para adquirir los siguientes ...,Orden de compra para adquirir los siguientes ...,Orden de compra para adquirir los siguientes ...
method,direct,direct,direct,direct,direct
suppliers,KRONOS LABORATORIOS C. LTDA.,KRONOS LABORATORIOS C. LTDA.,LABORATORIOS H.G. C.A.,LABORATORIOS INDUSTRIALES FARMACEUTICOS ECUAT...,LABORATORIOS INDUSTRIALES FARMACEUTICOS ECUAT...
buyer,DIRECCION DISTRITAL 06D03-CUMANDA-PALLATANGA-...,DIRECCION DISTRITAL 06D03-CUMANDA-PALLATANGA-...,DIRECCION DISTRITAL 06D03-CUMANDA-PALLATANGA-...,DIRECCION DISTRITAL 06D03-CUMANDA-PALLATANGA-...,DIRECCION DISTRITAL 06D03-CUMANDA-PALLATANGA-...
amount,504.000000,376.000000,516.000000,136.000000,155.200000


### Memory

In [4]:
(df
 .memory_usage(deep=True)
 .apply(lambda s_: s_ / 1000000)
 .pipe(lambda df_: pd.concat([df_, df.dtypes], axis = 1))
 .rename(columns={0:'memory', 1:'dtype'})
)

,memory,dtype
Index,0.000128,NaN
year,12.495928,int64
ocid,146.111533,object
date,129.644349,object
region,104.275575,object
title,139.833061,object
description,256.879964,object
method,103.584980,object
suppliers,153.588971,object
buyer,169.779263,object


### Numeric Types

In [5]:
(df
 .loc[:, ['year', 'amount', 'budget']]
 .replace(to_replace=[' None'], value=np.nan) # ' None' data cell must be removed
 .assign(year=lambda df_:df_.year.astype('int16'),
         **{c: lambda df_, c=c:df_[c].astype('float32') for c in ['budget', 'amount']})
 .sample(10)
)

,year,amount,budget
915058,2019,110.543999,110.543999
540869,2018,1955.459961,1955.459961
757546,2018,0.840000,0.840000
1198980,2020,25.760000,25.760000
1326772,2021,2.016000,2.016000
564542,2018,46.535999,46.535999
1560369,2022,0.000000,142000.000000
646557,2018,899.989990,899.989990
265870,2017,19.813429,19.813429
1978,2016,544.000000,544.000000


In [6]:
for size in [np.float16, np.float32, np.float64]:
    print(f'{size}:{np.finfo(size)}')

<class 'numpy.float16'>:Machine parameters for float16
---------------------------------------------------------------
precision =   3   resolution = 1.00040e-03
machep =    -10   eps =        9.76562e-04
negep =     -11   epsneg =     4.88281e-04
minexp =    -14   tiny =       6.10352e-05
maxexp =     16   max =        6.55040e+04
nexp =        5   min =        -max
---------------------------------------------------------------

<class 'numpy.float32'>:Machine parameters for float32
---------------------------------------------------------------
precision =   6   resolution = 1.0000000e-06
machep =    -23   eps =        1.1920929e-07
negep =     -24   epsneg =     5.9604645e-08
minexp =   -126   tiny =       1.1754944e-38
maxexp =    128   max =        3.4028235e+38
nexp =        8   min =        -max
---------------------------------------------------------------

<class 'numpy.float64'>:Machine parameters for float64
---------------------------------------------------------------
p

### Non-Numeric Types

In [136]:
(df
 .loc[(~df
        .region
        .str
        .contains('^\s\d\d?')), 
       ['ocid', 'region', 
        'title', 'method', 
        'suppliers', 'buyer', 
        'description']]
 .replace(to_replace=[' None', 'None'], value=np.nan)
 .applymap(lambda tx: tx.strip().title() if isinstance(tx, str) else tx)
 .assign(region=lambda df_:(df_
                            .region
                            .astype('category')),
         description=lambda df_:(df
                                 .description
                                 .replace(to_replace=['Orden De Compra Para Adquirir Los Siguientes Productos:\s?\s?', '^\s?\s?\?.?.dquisici.n Del?\s'], 
                                          value='', 
                                          regex=True)
                                 .astype('category')),
         method=lambda df_:(df_
                            .method
                            .mask(cond=lambda s:~s.isin(['Direct', 'Selective', 'Open', 'Limited']))
                            .astype('category')),
         **{c:lambda df_, c=c:df_[c].str.strip().str.title() for c in ['suppliers', 'buyer']})
)

In [137]:
df1.title.drop_duplicates()

0          Orden De Compra Ce-20160000536478
1          Orden De Compra Ce-20160000536479
2          Orden De Compra Ce-20160000536480
3          Orden De Compra Ce-20160000536481
4          Orden De Compra Ce-20160000536482
                         ...                
1561977          Re-Csps-Unae-006-202-607247
1561978              Re-Gadcg-018-2022-89167
1561982    Orden De Compra Ce-20220002245568
1561985              Cots-Hgpt-001-2022-2418
1561986           Coto-Gadmch-002-2022-43412
Name: title, Length: 1207427, dtype: object

### Datetime Types

In [178]:
# datetime format is ISO 8601 
(df
 .where(cond=lambda df_:df_.date.str.len()==26)
 .assign(date=lambda df_:pd.to_datetime(df_.date.str[:-6]))
)

,year,ocid,date,region,title,description,method,suppliers,buyer,amount,budget
0,2016.0,ocds-5wno2w-CE-20160000536478-586277,2016-05-16 00:00:00,CHIMBORAZO,Orden de compra CE-20160000536478,Orden de compra para adquirir los siguientes ...,direct,KRONOS LABORATORIOS C. LTDA.,DIRECCION DISTRITAL 06D03-CUMANDA-PALLATANGA-...,504.000000,504
1,2016.0,ocds-5wno2w-CE-20160000536479-586277,2016-05-16 00:00:00,CHIMBORAZO,Orden de compra CE-20160000536479,Orden de compra para adquirir los siguientes ...,direct,KRONOS LABORATORIOS C. LTDA.,DIRECCION DISTRITAL 06D03-CUMANDA-PALLATANGA-...,376.000000,376
2,2016.0,ocds-5wno2w-CE-20160000536480-586277,2016-05-16 00:00:00,CHIMBORAZO,Orden de compra CE-20160000536480,Orden de compra para adquirir los siguientes ...,direct,LABORATORIOS H.G. C.A.,DIRECCION DISTRITAL 06D03-CUMANDA-PALLATANGA-...,516.000000,516
3,2016.0,ocds-5wno2w-CE-20160000536481-586277,2016-05-16 00:00:00,CHIMBORAZO,Orden de compra CE-20160000536481,Orden de compra para adquirir los siguientes ...,direct,LABORATORIOS INDUSTRIALES FARMACEUTICOS ECUAT...,DIRECCION DISTRITAL 06D03-CUMANDA-PALLATANGA-...,136.000000,136
4,2016.0,ocds-5wno2w-CE-20160000536482-586277,2016-05-16 00:00:00,CHIMBORAZO,Orden de compra CE-20160000536482,Orden de compra para adquirir los siguientes ...,direct,LABORATORIOS INDUSTRIALES FARMACEUTICOS ECUAT...,DIRECCION DISTRITAL 06D03-CUMANDA-PALLATANGA-...,155.200000,155.2
...,...,...,...,...,...,...,...,...,...,...,...
1561986,2022.0,ocds-5wno2w-COTO-GADMCH-002-2022-43412,2022-07-21 08:26:55,MANABI,COTO-GADMCH-002-2022-43412,RECONSTRUCCION DEL PARQUE DEL PADRE Y CONSTRU...,open,None,MUNICIPIO DE CHONE,245451.127300,248932.44
1561987,2022.0,ocds-5wno2w-LICS-CZ3-HBA-001-202-121258,2022-06-24 07:29:04,CHIMBORAZO,LICS-CZ3-HBA-001-202-121258,CONTRATACIÓN DEL SERVICIO DE PÓLIZAS DE SEGUR...,open,None,HOSPITAL BASICO ALAUSI,4267.150000,9448.29
1561988,2022.0,ocds-5wno2w-RE1-RA-GADCC-004-202-45403,2022-06-09 07:08:43,LOJA,RE1-RA-GADCC-004-202-45403,ADQUISICION DE REPUESTOS MENORES PARA EL ARRE...,direct,None,MUNICIPIO DE CALVAS,0.000000,23377.99
1561989,2022.0,ocds-5wno2w-RE-CEP-GZAR-01-2022-31595,2022-07-15 08:34:31,EL ORO,RE-CEP-GZAR-01-2022-31595,TRABAJOS DE REMEDIACIÓN SUPERFICIAL Y DE GALE...,direct,None,GOBIERNO AUTONOMO DESCENTRALIZADO MUNICIPAL D...,1600512.880000,1600512.88


In [159]:
len('2018-06-01T00:00:00-05:00')

25

### Duplicates

In [179]:
# 346875 duplicates found and removed
def get_shape(df):
    print(df.shape)
    return df

(df
 .pipe(get_shape)
 .drop_duplicates()
 .pipe(get_shape)
)

(1561991, 11)
(1215116, 11)


,year,ocid,date,region,title,description,method,suppliers,buyer,amount,budget
0,2016,ocds-5wno2w-CE-20160000536478-586277,2016-05-16T00:00:00-05:00,CHIMBORAZO,Orden de compra CE-20160000536478,Orden de compra para adquirir los siguientes ...,direct,KRONOS LABORATORIOS C. LTDA.,DIRECCION DISTRITAL 06D03-CUMANDA-PALLATANGA-...,504.000000,504
1,2016,ocds-5wno2w-CE-20160000536479-586277,2016-05-16T00:00:00-05:00,CHIMBORAZO,Orden de compra CE-20160000536479,Orden de compra para adquirir los siguientes ...,direct,KRONOS LABORATORIOS C. LTDA.,DIRECCION DISTRITAL 06D03-CUMANDA-PALLATANGA-...,376.000000,376
2,2016,ocds-5wno2w-CE-20160000536480-586277,2016-05-16T00:00:00-05:00,CHIMBORAZO,Orden de compra CE-20160000536480,Orden de compra para adquirir los siguientes ...,direct,LABORATORIOS H.G. C.A.,DIRECCION DISTRITAL 06D03-CUMANDA-PALLATANGA-...,516.000000,516
3,2016,ocds-5wno2w-CE-20160000536481-586277,2016-05-16T00:00:00-05:00,CHIMBORAZO,Orden de compra CE-20160000536481,Orden de compra para adquirir los siguientes ...,direct,LABORATORIOS INDUSTRIALES FARMACEUTICOS ECUAT...,DIRECCION DISTRITAL 06D03-CUMANDA-PALLATANGA-...,136.000000,136
4,2016,ocds-5wno2w-CE-20160000536482-586277,2016-05-16T00:00:00-05:00,CHIMBORAZO,Orden de compra CE-20160000536482,Orden de compra para adquirir los siguientes ...,direct,LABORATORIOS INDUSTRIALES FARMACEUTICOS ECUAT...,DIRECCION DISTRITAL 06D03-CUMANDA-PALLATANGA-...,155.200000,155.2
...,...,...,...,...,...,...,...,...,...,...,...
1561977,2022,ocds-5wno2w-RE-CSPS-UNAE-006-202-607247,2022-08-04T08:57:28-05:00,CAÑAR,RE-CSPS-UNAE-006-202-607247,Contratación de servicios comunicacionales pa...,limited,None,UNIVERSIDAD NACIONAL DE EDUCACION UNAE,50050.000000,55672.11
1561978,2022,ocds-5wno2w-RE-GADCG-018-2022-89167,2022-07-30T08:24:05-05:00,BOLIVAR,RE-GADCG-018-2022-89167,SERVICIO DE DIFUSIÓN PARA EL MEJORAMIENTO DE ...,direct,None,GOBIERNO AUTÓNOMO DESCENTRALIZADO DEL CANTÓN ...,3200.000000,3200
1561982,2022,ocds-5wno2w-CE-20220002245568-43246,2022-07-26T00:00:00-05:00,PICHINCHA,Orden de compra CE-20220002245568,Orden de compra para adquirir los siguientes ...,direct,PHARMABRAND S.A.,HOSPITAL DE ESPECIALIDADES FUERZAS ARMADAS NO. 1,2.478000,2.478
1561985,2022,ocds-5wno2w-COTS-HGPT-001-2022-2418,2022-07-08T07:43:41-05:00,TUNGURAHUA,COTS-HGPT-001-2022-2418,PROVISIÓN DE COMBUSTIBLE PARA LOS VEHÍCULOS M...,open,None,GOBIERNO PROVINCIAL DE TUNGURAHUA,278568.080000,278568.08


### Missing Values

In [185]:
(df
 .isna()
 .groupby(df.year)
 .mean()
 .style
 .format(lambda n: '{:.4f}'.format(n*100))
)

,year,ocid,date,region,title,description,method,suppliers,buyer,amount,budget
year,,,,,,,,,,,
2016,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0004,0.0000,0.0000,0.0000
2017,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2018,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0006,0.0000,0.0000,0.0000
2019,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0713,0.0000,0.0000,0.0000
2020,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0597,0.0000,0.0000,0.0000
2021,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0280,0.0000,0.0000,0.0000
2022,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0348,0.0000,0.0000,0.0000


In [186]:
(df
 .isna()
 .groupby(df.region)
 .mean()
 .style
 .format(lambda n: '{:.4f}'.format(n*100))
)

,year,ocid,date,region,title,description,method,suppliers,buyer,amount,budget
region,,,,,,,,,,,
2016-10-03T00:00:00-05:00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2017-02-20T00:00:00-05:00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2017-05-24T00:00:00-05:00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2017-06-08T00:00:00-05:00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2017-07-18T00:00:00-05:00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2017-09-28T00:00:00-05:00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2017-10-10T00:00:00-05:00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2017-10-11T00:00:00-05:00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2017-10-26T00:00:00-05:00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


### Tweak Contrataciones

In [208]:
def tweak_contrataciones(df):
    return (df
            .drop_duplicates()
            .drop(columns='ocid')
            .loc[(~df.region.str.contains('^\s\d\d?')),]
            .applymap(lambda tx: tx.strip().title() if isinstance(tx, str) else tx)
            .replace(to_replace=['None'], value=np.nan)
            .assign(year=lambda df_:df_.year.astype('int16'),
                    **{c: lambda df_, c=c:df_[c].astype('float32') for c in ['budget', 'amount']},
                    region=lambda df_:(df_
                                       .region
                                       .astype('category')),
                    description=lambda df_:(df
                                            .description
                                            .replace(to_replace=['Orden De Compra Para Adquirir Los Siguientes Productos:\s?\s?', '^\s?\s?\?.?.dquisici.n Del?\s'], 
                                                     value='', 
                                                     regex=True)
                                            .astype('category')),
                    method=lambda df_:(df_
                                       .method
                                       .mask(cond=lambda s:~s.isin(['Direct', 'Selective', 'Open', 'Limited']))
                                       .astype('category')),
                    **{c:lambda df_, c=c:df_[c].str.strip().str.title() for c in ['suppliers', 'buyer']}
                   )
            .set_index('date')
           )

In [209]:
df_cc = tweak_contrataciones(df)

In [210]:
df_cc

,year,region,title,description,method,suppliers,buyer,amount,budget
date,,,,,,,,,
2016-05-16T00:00:00-05:00,2016,Chimborazo,Orden De Compra Ce-20160000536478,Orden de compra para adquirir los siguientes ...,Direct,Kronos Laboratorios C. Ltda.,Direccion Distrital 06D03-Cumanda-Pallatanga-S...,504.000000,504.000000
2016-05-16T00:00:00-05:00,2016,Chimborazo,Orden De Compra Ce-20160000536479,Orden de compra para adquirir los siguientes ...,Direct,Kronos Laboratorios C. Ltda.,Direccion Distrital 06D03-Cumanda-Pallatanga-S...,376.000000,376.000000
2016-05-16T00:00:00-05:00,2016,Chimborazo,Orden De Compra Ce-20160000536480,Orden de compra para adquirir los siguientes ...,Direct,Laboratorios H.G. C.A.,Direccion Distrital 06D03-Cumanda-Pallatanga-S...,516.000000,516.000000
2016-05-16T00:00:00-05:00,2016,Chimborazo,Orden De Compra Ce-20160000536481,Orden de compra para adquirir los siguientes ...,Direct,Laboratorios Industriales Farmaceuticos Ecuato...,Direccion Distrital 06D03-Cumanda-Pallatanga-S...,136.000000,136.000000
2016-05-16T00:00:00-05:00,2016,Chimborazo,Orden De Compra Ce-20160000536482,Orden de compra para adquirir los siguientes ...,Direct,Laboratorios Industriales Farmaceuticos Ecuato...,Direccion Distrital 06D03-Cumanda-Pallatanga-S...,155.199997,155.199997
...,...,...,...,...,...,...,...,...,...
2022-08-04T08:57:28-05:00,2022,Cañar,Re-Csps-Unae-006-202-607247,Contratación de servicios comunicacionales pa...,Limited,NaN,Universidad Nacional De Educacion Unae,50050.000000,55672.109375
2022-07-30T08:24:05-05:00,2022,Bolivar,Re-Gadcg-018-2022-89167,SERVICIO DE DIFUSIÓN PARA EL MEJORAMIENTO DE ...,Direct,NaN,Gobierno Autónomo Descentralizado Del Cantón G...,3200.000000,3200.000000
2022-07-26T00:00:00-05:00,2022,Pichincha,Orden De Compra Ce-20220002245568,Orden de compra para adquirir los siguientes ...,Direct,Pharmabrand S.A.,Hospital De Especialidades Fuerzas Armadas No. 1,2.478000,2.478000


### Missing values after tweaks

In [211]:
(df_cc
 .isna()
 .groupby(df_cc.year)
 .mean()
 .style
 .format(lambda n: '{:.4f}'.format(n*100))
)

,year,region,title,description,method,suppliers,buyer,amount,budget
year,,,,,,,,,
2016,0.0000,1.2217,0.0006,0.0000,1.9120,0.0577,0.0000,0.0546,4.4893
2017,0.0000,1.9980,0.0003,0.0000,2.7954,0.0323,0.0000,0.0316,2.9727
2018,0.0000,2.0304,0.0000,0.0000,2.4272,0.0195,0.0000,0.0184,3.8549
2019,0.0000,1.8121,0.0000,0.0000,2.7181,0.0967,0.0000,0.0312,6.5523
2020,0.0000,0.2926,0.0000,0.0000,2.5463,0.7037,0.0000,0.0333,5.0892
2021,0.0000,0.1813,0.6745,0.0000,3.3523,12.4380,0.0000,1.1900,15.8652
2022,0.0000,0.1712,8.1940,0.0000,10.8308,34.0359,0.0000,8.2433,22.3090


In [212]:
(df_cc
 .isna()
 .groupby(df_cc.region)
 .mean()
 .style
 .format(lambda n: '{:.4f}'.format(n*100))
)

,year,region,title,description,method,suppliers,buyer,amount,budget
region,,,,,,,,,
Azuay,0.0000,0.0000,0.9409,0.0000,4.1571,4.9454,0.0000,1.0632,8.1111
Bolivar,0.0000,0.0000,0.4131,0.0000,1.7901,2.3905,0.0000,0.4682,4.6544
Carchi,0.0000,0.0000,0.5442,0.0000,0.6959,3.5819,0.0000,0.5844,4.2778
Cañar,0.0000,0.0000,0.5062,0.0000,3.4449,3.2210,0.0000,0.5706,5.1781
Chimborazo,0.0000,0.0000,0.5520,0.0000,1.4592,3.6691,0.0000,0.6543,6.4892
Cotopaxi,0.0000,0.0000,0.6311,0.0000,1.0923,3.5926,0.0000,0.7070,6.3446
El Oro,0.0000,0.0000,0.4201,0.0000,0.7807,2.5928,0.0000,0.4616,4.7384
Esmeraldas,0.0000,0.0000,0.6147,0.0000,1.9912,3.3777,0.0000,0.7550,5.5158
Galapagos,0.0000,0.0000,0.6540,0.0000,0.9129,3.6790,0.0000,0.8039,7.9984


### Saving finishes pd.DataFrame to csv file

In [213]:
df_cc.to_csv('../data/contratacion_db.csv')

### Anomalies

#### Region column contains estranged values

In [48]:
print((df
 .region
 .unique()
 .tolist()
))

(df
 [(df
    .region
    .str
    .contains('^\s\d\d?'))]
 .head()
)

[' CHIMBORAZO', ' PICHINCHA', ' None', ' GALAPAGOS', ' TUNGURAHUA', ' LOJA', ' CAÑAR', ' GUAYAS', ' ORELLANA', ' AZUAY', ' CARCHI', ' MANABI', ' SANTA ELENA', ' NAPO', ' IMBABURA', ' EL ORO', ' MORONA SANTIAGO', ' BOLIVAR', ' LOS RIOS', ' SUCUMBIOS', ' SANTO DOMINGO DE LOS TSACHILAS', ' COTOPAXI', ' PASTAZA', ' ESMERALDAS', ' ZAMORA CHINCHIPE', ' 2016-10-03T00:00:00-05:00', ' 2017-05-24T00:00:00-05:00', ' 2017-06-08T00:00:00-05:00', ' 2017-11-15T00:00:00-05:00', ' 2017-02-20T00:00:00-05:00', ' 2017-09-28T00:00:00-05:00', ' 2017-10-11T00:00:00-05:00', ' 2017-07-18T00:00:00-05:00', ' 2017-10-10T00:00:00-05:00', ' 2017-10-26T00:00:00-05:00', ' 2018-03-26T00:00:00-05:00', ' 2018-11-15T00:00:00-05:00', ' 2018-05-03T00:00:00-05:00', ' 2018-05-07T00:00:00-05:00', ' 2018-03-28T00:00:00-05:00', ' 2019-02-07T00:00:00-05:00', ' 2019-10-01T00:00:00-05:00', ' 2019-12-03T00:00:00-05:00', ' 2020-09-08T00:00:00-05:00', ' 2020-12-01T00:00:00-05:00', ' 2020-03-23T00:00:00-05:00', ' 2020-07-17T00:00:00-0

,year,ocid,date,region,title,description,method,suppliers,buyer,amount,budget
206881,2016,ocds-5wno2w-MCO-GADPC -001-2016,-110462,2016-10-03T00:00:00-05:00,COTOPAXI,MCO-GADPC -001-2016-110462 Este procedimiento...,selective,CARDENAS SALAMEA DANIEL BOLIVAR,GOBIERNO AUTONOMO DESCENTRALIZADO PARROQUIAL ...,36993.410700,36993.4107
502076,2017,ocds-5wno2w-MCO-GADPC-001-2017,-110462,2017-05-24T00:00:00-05:00,COTOPAXI,MCO-GADPC-001-2017-110462 “ REFACCIONAMIENTO ...,selective,TAIPE CONDORCANA LUIS ERNESTO,GOBIERNO AUTONOMO DESCENTRALIZADO PARROQUIAL ...,19647.607800,19647.6078
504233,2017,ocds-5wno2w-SIE-MINTEL-004-2017,-236348,2017-06-08T00:00:00-05:00,PICHINCHA,SIE-MINTEL-004-2017-236348 ADQUISICIÓN DE TON...,open,MUÑOZ BRAVO FREDDY ARTURO,MINISTERIO DE TELECOMUNICACIONES Y DE LA SOCI...,47732.000000,None
506380,2017,ocds-5wno2w-MCO-0001-GAPRP-2017,-98128,2017-11-15T00:00:00-05:00,TUNGURAHUA,MCO-0001-GAPRP-2017-98128 REMASILLADO DE LAS ...,selective,None,GOBIERNO AUTONOMO DESCENTRALIZADO PARROQUIAL ...,None,18576.0268
513536,2017,ocds-5wno2w-CDC-GADPRC-001-2017,-110462,2017-02-20T00:00:00-05:00,COTOPAXI,CDC-GADPRC-001-2017-110462 ELABORACION DEL PR...,direct,PLAZARTE AGAMA ANGEL EUSTORGIO,GOBIERNO AUTONOMO DESCENTRALIZADO PARROQUIAL ...,2441.980000,2441.98


#### The Method column contains estranged values that must be removed

In [134]:
(df
 .method
 .value_counts()
 .pipe(lambda s: (s / s.sum()) * 100)
)

 selective                                                       51.841720
 direct                                                          37.539205
 open                                                             7.203819
 None                                                             0.640209
 COMPAÑIA GENERAL DE COMERCIO COGECOMSA S. A.                     0.455636
                                                                   ...    
 ASOCIACIÓN DE SERVICIOS DE LIMPIEZA AMBALIMPIEZA " ASOAMBA"      0.000064
 AÑAZCO ZAMBRANO KATHIUSKA MISHEL                                 0.000064
 ASOCIACION DE PRODUCCION TEXTIL HILANDO SUEÑOS ASOPROTEXHISU     0.000064
 GILER ZAMBRANO CESAR ENRIQUE                                     0.000064
 BRIONEZ PROAÑO DANIELA ESTEFANIA                                 0.000064
Name: method, Length: 3153, dtype: float64

#### OCID and Title columns contain essentially the same information except for first two columns in OCID column after expansion

In [131]:
# the contents of both are a mystery
(df
 .ocid
 .str.split('-', expand=True)
 .iloc[:, 2:]
 .rename(columns=lambda n: n-2)
 .pipe(lambda df_: pd.DataFrame(df_.values, 
                                index=df_.index, 
                                columns=pd.MultiIndex.from_product([['OCID'], df_.columns])))
 .pipe(lambda df_ : pd.concat([df_, (df
                                     .title
                                     .replace(to_replace='Orden de compra ', value='', regex=True)
                                     .str.split('-', expand=True)
                                     .pipe(lambda df_: pd.DataFrame(df_.values, index=df_.index, columns=pd.MultiIndex.from_product([['TITLE'], df_.columns])))
                                    )], 
                              axis=1))
)

OCID                                                           TITLE  \
            0               1       2     3      4       5     6     7      0   
0          CE  20160000536478  586277  None   None    None  None  None     CE   
1          CE  20160000536479  586277  None   None    None  None  None     CE   
2          CE  20160000536480  586277  None   None    None  None  None     CE   
3          CE  20160000536481  586277  None   None    None  None  None     CE   
4          CE  20160000536482  586277  None   None    None  None  None     CE   
...       ...             ...     ...   ...    ...     ...   ...   ...    ...   
1561986  COTO          GADMCH     002  2022  43412    None  None  None   COTO   
1561987  LICS             CZ3     HBA   001    202  121258  None  None   LICS   
1561988   RE1              RA   GADCC   004    202   45403  None  None    RE1   
1561989    RE             CEP    GZAR    01   2022   31595  None  None     RE   
1561990    RE            CSCD   EEASA   063     22    3233  None  None     RE   

                                                                 
                      1      2     3      4       5     6     7  
0        20160000536478   None  None   None    None  None  None  
1        20160000536479   None  None   None    None  None  None  
2        20160000536480   None  None   None    None  None  None  
3        20160000536481   None  None   None    None  None  None  
4        20160000536482   None  None   None    None  None  None  
...                 ...    ...   ...    ...     ...   ...   ...  
1561986          GADMCH    002  2022  43412    None  None  None  
1561987             CZ3    HBA   001    202  121258  None  None  
1561988              RA  GADCC   004    202   45403  None  None  
1561989             CEP   GZAR    01   2022   31595  None  None  
1561990            CSCD  EEASA   063     22    3233  None  None  

[1561991 rows x 16 columns]

#### API calls returning wrong data for columns fields

In [199]:
def get_shape(df):
    print(df.shape)
    return df
(df
 .loc[(df.region.str.contains('^\s\d\d?')),]
 .pipe(get_shape))

(48, 11)


,year,ocid,date,region,title,description,method,suppliers,buyer,amount,budget
206881,2016,ocds-5wno2w-MCO-GADPC -001-2016,-110462,2016-10-03T00:00:00-05:00,COTOPAXI,MCO-GADPC -001-2016-110462 Este procedimiento...,selective,CARDENAS SALAMEA DANIEL BOLIVAR,GOBIERNO AUTONOMO DESCENTRALIZADO PARROQUIAL ...,36993.410700,36993.4107
502076,2017,ocds-5wno2w-MCO-GADPC-001-2017,-110462,2017-05-24T00:00:00-05:00,COTOPAXI,MCO-GADPC-001-2017-110462 “ REFACCIONAMIENTO ...,selective,TAIPE CONDORCANA LUIS ERNESTO,GOBIERNO AUTONOMO DESCENTRALIZADO PARROQUIAL ...,19647.607800,19647.6078
504233,2017,ocds-5wno2w-SIE-MINTEL-004-2017,-236348,2017-06-08T00:00:00-05:00,PICHINCHA,SIE-MINTEL-004-2017-236348 ADQUISICIÓN DE TON...,open,MUÑOZ BRAVO FREDDY ARTURO,MINISTERIO DE TELECOMUNICACIONES Y DE LA SOCI...,47732.000000,None
506380,2017,ocds-5wno2w-MCO-0001-GAPRP-2017,-98128,2017-11-15T00:00:00-05:00,TUNGURAHUA,MCO-0001-GAPRP-2017-98128 REMASILLADO DE LAS ...,selective,None,GOBIERNO AUTONOMO DESCENTRALIZADO PARROQUIAL ...,None,18576.0268
513536,2017,ocds-5wno2w-CDC-GADPRC-001-2017,-110462,2017-02-20T00:00:00-05:00,COTOPAXI,CDC-GADPRC-001-2017-110462 ELABORACION DEL PR...,direct,PLAZARTE AGAMA ANGEL EUSTORGIO,GOBIERNO AUTONOMO DESCENTRALIZADO PARROQUIAL ...,2441.980000,2441.98
513537,2017,ocds-5wno2w-CDC-GADPRC-002-2017,-110462,2017-09-28T00:00:00-05:00,COTOPAXI,CDC-GADPRC-002-2017-110462 “EJECUCIÓN DE TALL...,direct,BENAVIDES LARA JAIME PATRICIO,GOBIERNO AUTONOMO DESCENTRALIZADO PARROQUIAL ...,7233.000000,7233
513538,2017,ocds-5wno2w-CDC-GADPRC-003-2017,-110462,2017-10-11T00:00:00-05:00,COTOPAXI,CDC-GADPRC-003-2017-110462 COMPLEMENTARIEDAD ...,direct,PRUNA TAPIA WASHINGTON NOE,GOBIERNO AUTONOMO DESCENTRALIZADO PARROQUIAL ...,7780.000000,7780
513841,2017,ocds-5wno2w-MCO-GADPC-002-2017,-110462,2017-07-18T00:00:00-05:00,COTOPAXI,MCO-GADPC-002-2017-110462 OBRAS VARIAS EN EL ...,selective,ILAQUICHE LICTA FAUSTO HERNAN,GOBIERNO AUTONOMO DESCENTRALIZADO PARROQUIAL ...,17738.189400,17738.1894
513995,2017,ocds-5wno2w-MCO-0001-GAPRP-2017,-98128,2017-11-15T00:00:00-05:00,TUNGURAHUA,MCO-0001-GAPRP-2017-98128 REMASILLADO DE LAS ...,selective,None,GOBIERNO AUTONOMO DESCENTRALIZADO PARROQUIAL ...,None,18576.0268
516712,2017,ocds-5wno2w-MCO-AZLD-012-2017,-813053,2017-10-10T00:00:00-05:00,PICHINCHA,MCO-AZLD-012-2017-813053 ASFALTADO DE LA CALL...,selective,MOSCOSO MONTAÑO JACQUES WILLIAM,Municipio del Distrito Metropolitano de Quito,135158.140000,135158.14
